# Monte Carlo Simulation Analysis

## Overview
- **What this notebook does:** Demonstrates the high-performance Monte Carlo engine with convergence monitoring (R-hat, ESS, MCSE), ruin probability estimation under multiple insurance scenarios, and key simulation diagnostics.
- **Prerequisites:** [core/01_loss_distributions.ipynb](01_loss_distributions.ipynb), [core/02_insurance_structures.ipynb](02_insurance_structures.ipynb)
- **Estimated runtime:** 2--5 minutes
- **Audience:** [Practitioner]

## Why Monte Carlo Simulation?
Analytical formulas break down for realistic insurance programs with non-linear features (deductibles, limits, aggregate caps). Monte Carlo simulation lets us estimate any risk metric -- ruin probability, VaR, growth-rate distributions -- by running thousands of independent scenarios. This notebook shows how to configure the engine, verify convergence, and interpret results.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/core'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import time
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance import ManufacturerConfig, InsuranceProgram, EnhancedInsuranceLayer
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.monte_carlo import MonteCarloEngine, MonteCarloConfig
from ergodic_insurance.convergence import ConvergenceDiagnostics
from ergodic_insurance.visualization import WSJ_COLORS

pio.templates.default = "plotly_white"

# Reproducibility
np.random.seed(42)

## Configuration

In [ ]:
# Simulation size
N_SIMULATIONS = 1_000
N_YEARS = 10
N_CHAINS = 4

# Business parameters
INITIAL_ASSETS = 10_000_000
ASSET_TURNOVER = 0.5
OPERATING_MARGIN = 0.08
TAX_RATE = 0.25
RETENTION_RATIO = 0.8

# Loss parameters
ATTRITIONAL_PARAMS = {
    'base_frequency': 5.0,
    'severity_mean': 50_000,
    'severity_cv': 0.8,
}
LARGE_PARAMS = {
    'base_frequency': 0.5,
    'severity_mean': 2_000_000,
    'severity_cv': 1.2,
}
CATASTROPHIC_PARAMS = {
    'base_frequency': 0.02,
    'severity_xm': 10_000_000,
    'severity_alpha': 2.5,
}

# Default insurance layers
DEFAULT_LAYERS = [
    EnhancedInsuranceLayer(0, 5_000_000, 0.015),
    EnhancedInsuranceLayer(5_000_000, 20_000_000, 0.008),
    EnhancedInsuranceLayer(25_000_000, 25_000_000, 0.004),
]

print("Monte Carlo simulation configured.")

## 1. Engine Setup and Basic Run

The `MonteCarloEngine` orchestrates loss generation, insurance recovery, and manufacturer accounting across thousands of independent simulations.

In [ ]:
manufacturer_config = ManufacturerConfig(
    initial_assets=INITIAL_ASSETS,
    asset_turnover_ratio=ASSET_TURNOVER,
    base_operating_margin=OPERATING_MARGIN,
    tax_rate=TAX_RATE,
    retention_ratio=RETENTION_RATIO,
)
manufacturer = WidgetManufacturer(manufacturer_config)

loss_generator = ManufacturingLossGenerator(
    attritional_params=ATTRITIONAL_PARAMS,
    large_params=LARGE_PARAMS,
    catastrophic_params=CATASTROPHIC_PARAMS,
    seed=42,
)

insurance_program = InsuranceProgram(DEFAULT_LAYERS)

config = MonteCarloConfig(
    n_simulations=N_SIMULATIONS,
    n_years=N_YEARS,
    n_chains=N_CHAINS,
    parallel=False,
    progress_bar=True,
    seed=42,
)

engine = MonteCarloEngine(
    loss_generator=loss_generator,
    insurance_program=insurance_program,
    manufacturer=manufacturer,
    config=config,
)

print(f"Engine configured: {config.n_simulations:,} simulations x {config.n_years} years")
print(f"Parallel: {config.parallel}, Chains: {config.n_chains}")

start_time = time.time()
results = engine.run()
elapsed = time.time() - start_time

# ruin_probability is a dict keyed by year string
ruin_prob = results.ruin_probability.get(str(N_YEARS), 0.0)

print(f"\nCompleted in {elapsed:.1f}s ({N_SIMULATIONS / elapsed:.0f} sims/s)")
print(f"Ruin probability: {ruin_prob * 100:.2f}%")
print(f"Mean final assets: ${np.mean(results.final_assets):,.0f}")
print(f"Mean growth rate: {np.mean(results.growth_rates) * 100:.2f}%")

## 2. Convergence Monitoring

Reliable Monte Carlo estimates require convergence diagnostics. We track three metrics:
- **R-hat:** Ratio of between-chain to within-chain variance (target < 1.1)
- **ESS:** Effective Sample Size after accounting for autocorrelation
- **MCSE:** Monte Carlo Standard Error of the mean estimate

In [ ]:
diagnostics = ConvergenceDiagnostics()

# Monitor convergence at increasing sample sizes
check_points = np.linspace(100, N_SIMULATIONS, 10).astype(int)
r_hats, ess_values, mcse_values = [], [], []

for n in check_points:
    chain_size = n // N_CHAINS
    if chain_size < 2:
        continue
    chains = results.growth_rates[:n].reshape(N_CHAINS, -1)
    r_hat = diagnostics.calculate_r_hat(chains)
    ess = diagnostics.calculate_ess(results.growth_rates[:n])
    mcse = diagnostics.calculate_mcse(results.growth_rates[:n], ess)
    r_hats.append(r_hat)
    ess_values.append(ess)
    mcse_values.append(mcse)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('R-hat Convergence', 'Effective Sample Size', 'Monte Carlo Standard Error'),
)

fig.add_trace(go.Scatter(
    x=check_points[:len(r_hats)], y=r_hats, mode='lines+markers',
    line=dict(color=WSJ_COLORS['blue'], width=2)), row=1, col=1)
fig.add_hline(y=1.1, line_dash='dash', line_color=WSJ_COLORS['red'],
              annotation_text='Target = 1.1', row=1, col=1)

fig.add_trace(go.Scatter(
    x=check_points[:len(ess_values)], y=ess_values, mode='lines+markers',
    line=dict(color=WSJ_COLORS['green'], width=2)), row=1, col=2)

fig.add_trace(go.Scatter(
    x=check_points[:len(mcse_values)], y=mcse_values, mode='lines+markers',
    line=dict(color=WSJ_COLORS['orange'], width=2)), row=1, col=3)

fig.update_layout(height=350, showlegend=False, title_text='Convergence Diagnostics')
fig.update_xaxes(title_text='Iterations')
fig.show()

print(f"Final R-hat: {r_hats[-1]:.3f} {'(converged)' if r_hats[-1] < 1.1 else '(not converged)'}")
print(f"Final ESS: {ess_values[-1]:.0f}")
print(f"Final MCSE: {mcse_values[-1]:.6f}")

## 3. Results Distribution

Examine the distribution of final assets, growth rates, and annual loss vs. recovery patterns.

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Final Assets Distribution', 'Growth Rate Distribution',
        'Annual Loss vs Recovery', 'Key Metrics',
    ),
    specs=[
        [{'type': 'histogram'}, {'type': 'histogram'}],
        [{'type': 'scatter'}, {'type': 'table'}],
    ],
)

fig.add_trace(go.Histogram(
    x=results.final_assets, nbinsx=50,
    marker_color=WSJ_COLORS['blue']), row=1, col=1)

fig.add_trace(go.Histogram(
    x=results.growth_rates * 100, nbinsx=50,
    marker_color=WSJ_COLORS['green']), row=1, col=2)

avg_annual_losses = results.annual_losses.mean(axis=0)
avg_annual_recoveries = results.insurance_recoveries.mean(axis=0)
years = list(range(1, N_YEARS + 1))

fig.add_trace(go.Scatter(
    x=years, y=avg_annual_losses, mode='lines+markers', name='Avg Loss',
    line=dict(color=WSJ_COLORS['red'], width=2)), row=2, col=1)
fig.add_trace(go.Scatter(
    x=years, y=avg_annual_recoveries, mode='lines+markers', name='Avg Recovery',
    line=dict(color=WSJ_COLORS['blue'], width=2)), row=2, col=1)

metrics_data = [
    ['Ruin Probability', f'{ruin_prob * 100:.2f}%'],
    ['Mean Final Assets', f'${np.mean(results.final_assets):,.0f}'],
    ['Mean Growth Rate', f'{np.mean(results.growth_rates) * 100:.2f}%'],
    ['Std Growth Rate', f'{np.std(results.growth_rates) * 100:.2f}%'],
    ['Simulations', f'{N_SIMULATIONS:,}'],
    ['Execution Time', f'{elapsed:.1f}s'],
]

fig.add_trace(go.Table(
    header=dict(values=['Metric', 'Value'], align='left'),
    cells=dict(values=list(zip(*metrics_data)), align='left'),
), row=2, col=2)

fig.update_layout(height=700, showlegend=True, title_text='Monte Carlo Simulation Results')
fig.update_xaxes(title_text='Final Assets', row=1, col=1, tickformat='$.2s')
fig.update_xaxes(title_text='Growth Rate (%)', row=1, col=2)
fig.update_xaxes(title_text='Year', row=2, col=1)
fig.update_yaxes(title_text='Amount', row=2, col=1, tickformat='$.2s')
fig.show()

## 4. Ruin Probability Across Insurance Scenarios

Compare ruin probabilities for no insurance, basic coverage, and a full three-layer program.

In [ ]:
insurance_scenarios = {
    'No Insurance': InsuranceProgram(layers=[]),
    'Basic Coverage': InsuranceProgram(layers=[
        EnhancedInsuranceLayer(0, 5_000_000, 0.02),
    ]),
    'Full Program': InsuranceProgram(layers=DEFAULT_LAYERS),
}

scenario_results = {}
for name, program in insurance_scenarios.items():
    eng = MonteCarloEngine(
        loss_generator=loss_generator,
        insurance_program=program,
        manufacturer=WidgetManufacturer(manufacturer_config),
        config=MonteCarloConfig(
            n_simulations=N_SIMULATIONS, n_years=N_YEARS,
            parallel=False, progress_bar=False, seed=42,
        ),
    )
    res = eng.run()
    scenario_results[name] = res
    res_ruin = res.ruin_probability.get(str(N_YEARS), 0.0)
    print(f"{name:20s} | Ruin: {res_ruin * 100:6.2f}% | "
          f"Mean assets: ${np.mean(res.final_assets):>12,.0f} | "
          f"Growth: {np.mean(res.growth_rates) * 100:+.2f}%")

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Ruin Probability by Scenario', 'Growth Rate Distributions'),
)

names = list(scenario_results.keys())
ruin_probs = [scenario_results[n].ruin_probability.get(str(N_YEARS), 0.0) * 100 for n in names]
colors = [WSJ_COLORS['red'], WSJ_COLORS['orange'], WSJ_COLORS['green']]

fig.add_trace(go.Bar(x=names, y=ruin_probs, marker_color=colors), row=1, col=1)

for name, color in zip(names, colors):
    fig.add_trace(go.Histogram(
        x=scenario_results[name].growth_rates * 100,
        name=name, opacity=0.6, nbinsx=40,
        marker_color=color), row=1, col=2)

fig.update_layout(height=400, title_text='Insurance Scenario Comparison', barmode='overlay')
fig.update_yaxes(title_text='Ruin Probability (%)', row=1, col=1)
fig.update_xaxes(title_text='Growth Rate (%)', row=1, col=2)
fig.show()

## Key Takeaways

- The Monte Carlo engine efficiently simulates thousands of independent business paths with loss generation and insurance recovery.
- Convergence diagnostics (R-hat, ESS, MCSE) confirm that simulation estimates are stable and reliable.
- Insurance coverage dramatically reduces ruin probability without proportionally reducing mean growth.
- The full three-layer program provides the best trade-off between cost and risk reduction.

## Next Steps

- **Quantify tail risk:** [core/05_risk_metrics.ipynb](05_risk_metrics.ipynb)
- **Study long-term dynamics:** [core/06_long_term_dynamics.ipynb](06_long_term_dynamics.ipynb)
- **Optimize retention levels:** [optimization/01_retention_optimization.ipynb](../optimization/01_retention_optimization.ipynb)